# Metodologia Final - Avaliação na Base REDD / Residência 6

Com os resultados obtidos até aqui foi possível consolidar a metodologia final que será adotada na pesquisa, a qual é estruturada em:

1. Taxa Amostral e Janela flexível por Aparelho;
2. Gráfico de Recorrência como ferramenta de pré-processamento da série temporal;
3. Rede Neural Convolucional como modelo classificador;
4. Estratégia de treinamento do modelo:
    * Transfer Learning do Aparelho em Outras residências;
    * Treinamento utilizando Punição para o Desbalanceamento de classes;
    * Adoção de Parada Antecipada (em função de AUC, a fim de otimizar sensibilidade);
    * Uso de Threshold-adaptativo;
    * Avaliação de Carga Descartada (threshold).
    
Sendo assim, neste estudo iremos aplicar a metodologia nos dados da base REDD, especificamente na `residência 6`.


# Configurações

In [1]:
import os
import sys
import gc
import json
from pprint import pprint
from collections import Counter
import copy
import warnings
warnings.filterwarnings(action="ignore")


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from tqdm import *

from pretty_confusion_matrix import *

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS
RESIDENCIA = 6

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "20", f"residencia_{RESIDENCIA}") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

# Dados

## Base REDD

In [3]:
# Gerar arquivo H5 (Nilmtk) do dataset REDD, caso n exista
if not os.path.isfile(arquivo_dataset):
    from nilmtk.dataset_converters import convert_redd
    
    print("Gerando arquivo H5 (NILMTK) da base REDD, aguarde...")
    print("-----")
    convert_redd(caminho_redd, arquivo_dataset)

# Carregando dataset REDD no objeto NILMTK
# Exemplo de carregamento da base REDD no NILMTK
import h5py # * Evitar erro de incompatibilidade entre h5py e nilmtk
from nilmtk import DataSet
from nilmtk.utils import print_dict
redd = DataSet(arquivo_dataset)
print("NILMTK -> Detalhes sobre o dataset REDD:")
print_dict(redd.metadata)
print()

# Parametros dos dados
PARAMETROS_DATASET = {
    "base":redd,
    "id_residencia": RESIDENCIA,
    "inicio_intervalo":'2011-04-16 05:11:30',
    "fim_intervalo":'2011-04-23 08:43:26',
    "debug": False    
}
print("PARÂMETROS DO ESTUDO:")
pprint(PARAMETROS_DATASET)

NILMTK -> Detalhes sobre o dataset REDD:



PARÂMETROS DO ESTUDO:
{'base': <nilmtk.dataset.DataSet object at 0x000001E949C9BA20>,
 'debug': False,
 'fim_intervalo': '2011-04-23 08:43:26',
 'id_residencia': 6,
 'inicio_intervalo': '2011-04-16 05:11:30'}


In [4]:
def carregar_dados_aparelho(janelas, instancia, aparelho, taxa, tamanho_janela, split_teste=None, eliminar_janelas_vazias=False, debug=False):
    # Extrair series divididas em janelas para cada medidor
    dados_cargas = janelas.preparar(
        taxa_amostral=taxa, 
        intervalo_medicao=tamanho_janela
    )
    print()

    # Pprearando dados (Serie / Estado)
    # X
    dados_medidores = janelas.filtrar_cargas(
        dados_cargas,
        filtros=[
            (1, 'site_meter'),
            (2, 'site_meter'),    
        ]
    )
    
    dados_aparelho = janelas.filtrar_cargas(dados_cargas, filtros=[(instancia, aparelho)])[0]
    
    # Validar tamanho dos dados de medidores (podem ter mais registros que os aparelhos)
    janela_media_medidores = int(np.sum([len(d["janelas"])for d in dados_medidores])/len(dados_medidores))
    janela_media_aparelho = len(dados_aparelho["janelas"])#int(np.sum([len(d["janelas"])for d in dados_aparelho])/len(dados_aparelho))

    # Ajustando para medidores terem o mesmo shape de janelas dos aparelhos 
    if janela_media_medidores > janela_media_aparelho:
        diferenca = janela_media_medidores-janela_media_aparelho
        #if debug: print("  -> Diferenca encontrada entre medidores/aparelhos:", diferenca, ", ajustando..")
        for i in range(len(dados_medidores)):
            removidos = 0
            while removidos < diferenca:
                # Remover ultima janela
                dados_medidores[i]["janelas"] = dados_medidores[i]["janelas"][:-1,:]
                removidos += 1
    
    # Estruturando dados modelagem (X e y)
    X = dados_medidores[0]["janelas"] + dados_medidores[1]["janelas"]

    # Selecionando apenas janelas VALIDAS (ocorrencia de ao menos 1 carga)
    # TODO: Implementar na biblioteca esta rotina de validacao
    if eliminar_janelas_vazias:
        idx_janelas_validas = np.where(np.sum(X, axis=1)>0)[0]
        X = X[idx_janelas_validas]
        #for i in range(len(dados_aparelhos)):
        dados_aparelho["janelas"] = dados_aparelho["janelas"][idx_janelas_validas]
        rotulos = copy.deepcopy(dados_aparelho["rotulos"])
        dados_aparelho["rotulos"]["estado"] = rotulos["estado"][idx_janelas_validas]
        dados_aparelho["rotulos"]["media"]  = rotulos["media"][idx_janelas_validas]
        dados_aparelho["rotulos"]["total"]  = rotulos["total"][idx_janelas_validas]
        if debug:
            print("   - `{}-{}`: {} => {}".format(
                dados_aparelho["carga"].upper(), 
                dados_aparelho["instancia"],
                Counter(rotulos["estado"]),
                Counter(dados_aparelho["rotulos"]["estado"])
            ))

    # y
    y = dados_aparelho["rotulos"]["estado"]

    # <<< Limpando memoria >>>
    dados_cargas = None
    del dados_cargas
    dados_medidores = None
    del dados_medidores
    dados_aparelho = None
    del dados_aparelho
    gc.collect()
    # <<< Limpando memoria >>>

    # Fazendo split dos dados (treino/teste)
    if split_teste is None:
        return X, y
    else:
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=split_teste,
            stratify=y,
            random_state=SEED
        )
        print()

        return X_treino, X_teste, y_treino, y_teste        
        

## Melhores Combinações de Taxas e Janelas para cada Aparelho (estudo 19)

In [5]:
df_melhores_taxas_janelas = pd.read_csv(os.path.join(caminho_dados, "19", "melhores_taxa_janela_aparelhos.csv"), index_col=0)
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela,loss,acuracia,precisao,recall,f1,f1_macro
0,sockets - 3,3,90,2.314749e-01,0.986014,40.00,66.67,50.00,74.65
1,sockets - 4,3,30,1.049300e-09,1.000000,0.00,0.00,0.00,100.00
2,light - 5,4,360,7.989106e-03,1.000000,0.00,0.00,0.00,100.00
3,ce_appliance - 6,3,30,1.013742e-09,1.000000,100.00,100.00,100.00,100.00
4,fridge - 7,8,1080,4.944706e-04,1.000000,100.00,100.00,100.00,100.00
5,waste_disposal_unit - 8,4,900,2.370523e+00,0.790698,11.11,50.00,18.18,53.09
6,dish_washer - 9,5,360,9.249995e-02,0.976744,50.00,50.00,50.00,74.40
7,electric_furnace - 10,5,720,1.154453e+00,0.906977,83.33,62.50,71.43,82.94
8,light - 11,8,900,2.041479e-01,0.909091,85.71,85.71,85.71,89.52
9,sockets - 12,5,540,2.085883e+00,0.859649,75.00,30.00,42.86,67.43


In [6]:
# TODO: 
# - Desenvolver módulo da metodologia na lib PyNILM

## Parâmetros de RP dos Aparelhos (estudo 18)

In [7]:
# Carregando arquivos de parametros, caso n estejam (kernel reiniciado)
if not 'parametros_rp_aparelho' in locals():
    with open(os.path.join(caminho_dados, "18", "parametros_rp_aparelho.json"),'r') as arquivo:
        parametros_rp_aparelho = json.load(arquivo)

# Metodologia
---

Aplicação da metodologia nos aparelhos.

## Ambiente e Funções Auxiliares

### Preparando ambiente de desenvolvimento 

In [8]:
%load_ext autoreload
%autoreload 2

# from PyNILM.dados.janelas import Janelas
from PyNILM.dados.janelas import Janelas
from PyNILM.avaliacao.metricas import *
from PyNILM.avaliacao.graficos import *
from PyNILM.avaliacao.analises import *

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from pyts.image import RecurrencePlot, GramianAngularField

import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import backend as K
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

# Garantindo reprodutibilidade
import random as rn
# Constantes dos experimentos
SEED = 33
FRACAO_TESTE = 0.25
EPOCAS = 100
TAMANHO_LOTE = 32
VERBOSIDADE = 2
# Parametros RP (verificado empiricamente)
PARAMETROS_RP = {
    "dimension": 1,
    "time_delay": 1,
    "threshold": None,
    "percentage": 10
}
TAMANHO_IMAGEM = (32,32)
# PARAMETROS_RP={
#     "dimension": 3,
#     "time_delay": 8,
#     "threshold": 0.1
# }
# Travar Seed's
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
np.random.seed(SEED)
rn.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

# Habilitando/limitando utilização de GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*3)]
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("# GPUs habilitadas:", "{} física(s)".format(len(gpus)), "/", "{} lógica(s)".format(len(gpus)))
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.

      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)

      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.

    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)

    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.mean(alpha * tf.pow(1. - pt_1, gamma) * tf.math.log(pt_1)) \
               -K.mean((1 - alpha) * tf.pow(pt_0, gamma) * tf.math.log(1. - pt_0))

    return binary_focal_loss_fixed


# GPUs habilitadas: 1 física(s) / 1 lógica(s)


### Rotinas da Etapa de Modelagem de DL

In [9]:
def convnet_metodologia(
    input_shape_ = (32, 32, 1), 
    output_dim=1, 
    optimizer='adam',
    loss_function = 'binary_crossentropy', 
    metrics=['accuracy'],
    output_activation = 'sigmoid',
    bias_output = None,
):
    
    if bias_output is not None:
        bias_output = tf.keras.initializers.Constant(bias_output)
        
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=input_shape_))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))    
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(output_dim, bias_initializer=bias_output, activation=output_activation))

    model.compile(optimizer=optimizer, loss=[loss_function], metrics=metrics)
    
    return model

In [10]:
def centralizar_dados(X):
    return np.array([x - x.mean() for x in X], dtype=TIPO_DADOS)

def normalizar_dados(X):
    X_ = np.empty(np.asarray(X).shape)
    for i, x in enumerate(X):
        if len(np.unique(x))>1:
            X_[i] = (x - x.min()) / (x.max() - x.min())
        elif x.max()>0:
            X_[i] = x / x.max()
        else:
            X_[i] = x
    return X_.astype(TIPO_DADOS)

def padronizar_dados(X):
    """
    Calcular z-score por amostra.
    Ref.: https://datascience.stackexchange.com/questions/16034/dtw-dynamic-time-warping-requires-prior-normalization    
    """
    from scipy import stats
    
    return np.array([stats.zscore(x) for x in X], dtype=TIPO_DADOS)

### Rotinas do Gráfico de Recorrências

In [11]:
# Construindo o pipeline de dados
# ----------

import cv2

# Constante fundamentais
TAMANHO_IMAGEM = (32,32,1) # Apenas 1 canal
TIPO_DADOS = np.float32
def serie_para_imagem(serie, params_rp = PARAMETROS_RP, tam_imagem=TAMANHO_IMAGEM, 
                      normalizar=False, padronizar=False):
    """
    Funcao responsavel por gerar e tratar a imagem RP (baseado estudo #17).
    """
    # Gerando imagem RP/redimensiona_prndo
    imagem = RecurrencePlot(**params_rp).fit_transform([serie])[0]
    imagem = cv2.resize(
            imagem, 
            dsize=tam_imagem[:2], 
            interpolation=cv2.INTER_CUBIC
        ).astype(TIPO_DADOS)
    
    if np.sum(imagem) > 0:
        # Normalizar
        if normalizar:
                imagem = (imagem - imagem.min()) / (imagem.max() - imagem.min()) # MinMax (0,1)
            #imagem = (imagem - imagem.mean()) / np.max([imagem.std(), 1e-4])

    #     # centralizar
    #     if centralizar:
    #         imagem -= imagem.mean()

        # Padronizar
        elif padronizar:
            imagem = (imagem - imagem.mean())/imagem.std()#tf.image.per_image_standardization(imagem).numpy()

    # N canais
    imagem = np.stack([imagem for i in range(tam_imagem[-1])],axis=-1).astype(TIPO_DADOS)     
    
    return imagem

def preparar_amostras(X, y, params_rp=PARAMETROS_RP, tam_imagem=TAMANHO_IMAGEM, normalizar=False, padronizar=False):
    X_imagem = np.empty((len(X), *TAMANHO_IMAGEM))
    for i, x in tqdm_notebook(enumerate(X), total=len(X)):
        X_imagem[i,] = serie_para_imagem(
            x, 
            params_rp=PARAMETROS_RP, 
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=normalizar,
            padronizar=padronizar,
        )
    return X_imagem, y

def preparar_amostra_tfdata(amostra, rotulo):
    """
    Preparação da amostra/rótulo para o modelo.
    """
    # Convertendo serie para imagem
    amostra = tf.numpy_function(serie_para_imagem, [amostra], TIPO_DADOS)
    amostra = tf.reshape(amostra, TAMANHO_IMAGEM)
    return amostra, rotulo

In [12]:
from nolitsa import delay, dimension, utils    

def obter_delay_autocorrelacao(x):
    # Compute autocorrelation and delayed mutual information.
    try:
        if np.unique(x).shape[0] > 1:
            r = delay.acorr(x, maxtau=x.shape[0])
            r_delay = np.argmax(r < 1.0 / np.e)
        else:
            r_delay = 1
    except:
        r_delay = 1
    finally:
        return r_delay

def dimensoes_validas(serie, dimensoes, delay, maxnum=None):
    """
    Baseado em:
        (linha 97 - https://github.com/manu-mannattil/nolitsa/blob/master/nolitsa/dimension.py); e
        (linha 165 - https://github.com/manu-mannattil/nolitsa/blob/master/nolitsa/utils.py)
    """
    
    
    if not maxnum:
        maxnum = (delay + 1) + 1 + (delay + 1)
    else:
        maxnum = max(1, maxnum)
        
    d = []
    comprimento_serie = len(serie)
    for dimensao in (dimensoes):
        # Reconstrucao do espaço de fases, se viavel
        try:
            y = utils.reconstruct(serie[:-delay], dimensao, delay)
            if maxnum < len(y):
                d.append(dimensao)
        except:
            pass
    return d

def obter_dimensao_falsosvizinhos(serie, dimensoes_avaliacao, delay, metrica="euclidean"):
    
    try: 
        f1, f2, f3 = dimension.fnn(
            serie, 
            tau=delay, 
            dim=dimensoes_avaliacao, 
            window=delay, 
            metric=metrica
        ) 
        embedding_dimension = f3.argmin()+1
    except:
        embedding_dimension = 1
    finally:
        return embedding_dimension
    
def k_valores_mais_frequentes(valores, k=10, ascendente=True):
    ordernado = sorted(Counter(valores).most_common(k), key=lambda tup: tup[0])
    counter_ordenado = sorted(ordernado, key=lambda tup: tup[1], reverse=True)
    return counter_ordenado

# Aplicação da Metodologia

In [13]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

janelas = Janelas(**PARAMETROS_DATASET)

def classificacao_threshold(modelo, X, y, threshold={0:0.5, 1:0.5}, grafico=True, debug=False):
    y_hat = modelo.predict(X_teste_imagem).round().astype(np.int16)
    y_proba = np.array([y[0] for y in modelo.predict(X_teste_imagem).round(3)])
    y_proba_ = np.array([1-proba if proba <= .5 else proba for proba in y_proba])

    # Graficos de analise
    if grafico:
        fig,axes = plt.subplots(2,2, figsize=(20,10));

        sns.distplot(y_proba, ax=axes[0,0], color='black')
        axes[0,0].set_title("Saída Sigmoid Modelo");

        sns.distplot(y_proba_, ax=axes[0,1], color="orange")
        axes[0,1].set_title("Probabilidades Escalanodas [0,1]");

        plt.suptitle("Distribuição de Probabilidades Inferidas", size=18, y=1.05);

        sns.distplot(y_proba_[y_teste==0], ax=axes[1,0], color='red')
        axes[1,0].set_title("Probabilidades p/ CLASSE `0`");

        sns.distplot(y_proba_[y_teste==1], ax=axes[1,1], color='g')
        axes[1,1].set_title("Probabilidades p/ CLASSE `1`");

        plt.tight_layout();
        plt.show()

    # Registros avaliados threshold p/ classe
    y_teste_ = []
    y_pred_ =  []
    idx_descartados = []
    
    for i, c in enumerate(threshold.keys()):
        
        # Selecionando registros por classe
        idx_classe = [y_[0] for y_ in np.argwhere(y_teste == c)]
        y_teste_classe = np.array(y_teste[idx_classe])
        y_pred_classe = np.array(y_hat[idx_classe])
        y_proba_classe = np.array(y_proba_[idx_classe])

        # Filtrando pela probabilidade da classe
        idx_threshold = [y_[0] for y_ in np.argwhere(y_proba_classe >= threshold[c])]
        idx_descartados_classe = list(
            set(range(len(y_proba_classe))) - set(idx_threshold)
        ) 
        
        y_teste_.extend(y_teste_classe[idx_threshold])
        y_pred_.extend(y_pred_classe[idx_threshold])
        if i == 0:
            idx_descartados.extend(idx_descartados_classe)
        else:
            idx_descartados.extend(np.array(idx_descartados_classe) + len(idx_classe))
    
    # Verificar o impacto do threshold
    suporte_relativo = (len(y_teste_)/len(y_teste))*100
    if debug:
        print("   - Análise de Suporte: {:.1f}% dos registros de teste considerados".format(
            suporte_relativo
        ) )
        print()

        print("   - Métricas:")
        print()
        
    acc = accuracy_score(y_teste_, y_pred_)
    f1_ = f1_score(y_teste_, y_pred_, average="macro")
    try:
        auc_ = roc_auc_score(y_teste_, y_pred_)
    except:
        if f1_ == 1:
            auc_ = 1
        else:
            auc_ = 0.5
    if debug:
        print("     _ Accuracy: {:.2f}%".format(acc))
        print("     _ F1-macro: {:.2f}%".format(f1_))
        print("     _ AUC     : {:.3f}".format(auc_))
        print()
        print("   - Relatório de classificação:")
        print()
        print(classification_report(y_teste_, y_pred_))
        print("   - Matrix de confusao:")
        print()
        print(confusion_matrix(y_teste_, y_pred_))

    return y_teste_, y_pred_, acc, f1_, auc_, suporte_relativo, idx_descartados

def calcular_carga_desconsiderada(y_pred, y_teste, X_teste, modo_calculo = "positivo"):
    if len(y_hat.shape)>1:
        y_pred = np.squeeze(y_pred)

    # Erros de classificacao
    idx_erros = [y_[0] for y_ in np.argwhere(np.equal(y_pred, y_teste) == False)]

    # Totalizar por erro cometido pelo modelo
    if modo_calculo == "positivo":
        total_por_erro = []
        for ie in idx_erros:
            if y_teste[ie] == 1:
                total_por_erro.append(np.sum(X_teste[ie]))
    elif modo_calculo == "negativo":
        total_por_erro = []
        for ie in idx_erros:
            if y_teste[ie] == 0:
                total_por_erro.append(np.sum(X_teste[ie]))
    else:
        total_por_erro = np.sum(X_teste[idx_erros], axis=1)
    return total_por_erro
    #np.sum(total_por_erro)

In [14]:
def instancia_aparelho_residencia(aparelho, residencia, base = redd):
    """Função para coletar o id/instancia do aparelho na residencia,
    permitindo executar os testes independente da residencia"""
    instancia = []
    #for e in base.buildings[residencia].elec.all_meters():
    for e_i in range(1, len(janelas.base.buildings[residencia].elec.all_meters())):

        # Selecionando canal/aparelho
        e = janelas.base.buildings[residencia].elec[e_i]
        
        if not hasattr(e,'meters'):
            if e.label().lower().replace(" ","_") == aparelho:
                instancia.append( e.instance() )
        else:
            for e_ in e.meters:
                if e_.label().lower().replace(" ","_") == aparelho:
                    instancia.append( e_.instance() )
    return instancia

## Metodologia Baseline

### Split Treino/Teste

In [32]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
    

    print(f"* Aparelho {CARGA.upper()}:")
    print()

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):
        
        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f} - instancia {})...".format(
            TAXA, TAMANHO_JANELA, INSTANCIA
        ))
        try:
            X, y = carregar_dados_aparelho(
                janelas=janelas,
                instancia=INSTANCIA,
                aparelho=CARGA,
                tamanho_janela=TAMANHO_JANELA,
                taxa=TAXA,
                eliminar_janelas_vazias=True
            )
            print()
            #break

            print("   - Detalhes da amostragem (lotes):")
            print("   ---")
            for item in Counter(y).items():
                print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
            print()

            # Checando series estaveis
            estavel = []
            for i, x in enumerate(X):
                if len(np.unique(x)) == 1:
                    estavel.append(i)
            print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
            print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
            print()

            print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
            print()

            # Convertendo series para imagem
            print("   - Preparando dados para modelagem (treino/teste)...")
            print("   ---")
            # Fazendo split dos dados (treino/teste)
            X_treino, X_teste, y_treino, y_teste = train_test_split(
                X, y, 
                test_size=FRACAO_TESTE,
                stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
                random_state=SEED
            )
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                ##################### METODOLOGIA ANTIGA #####################
                # Modelo baseline
                modelo = convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                )
                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando
                y_hat = modelo.predict(X_teste_imagem).round().astype(np.int16)
                ##################### METODOLOGIA ANTIGA #####################

                tf.keras.backend.clear_session()

                # Incrementando resultados
                acc = accuracy_score(y_teste, y_hat)
                score = f1_score(y_teste, y_hat, average="macro")
                try:
                    auc_score = roc_auc_score(y_teste, y_hat)
                except:
                    if score == 1:
                        auc_score = 1
                    else:
                        auc_score = 0.5
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste)
                y_pred.extend(y_hat)

                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)
        

            print()
            print("   - Resultados finais:")
            print("   ---")
            print()

            print("      -> Acurácia:")
            print()
            print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
            print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
            print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
            print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
            print()

            print("      -> F1-score (macro):")
            print()
            print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
            print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
            print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
            print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
            print()
            print("      -> AUC:")
            print()
            print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
            print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
            print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
            print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
            print()

            print("      -> Relatório de classificação:")
            print()
            print(classification_report(y_true, y_pred))
            print("      -> Matrix de confusao:")
            print()
            print(confusion_matrix(y_true, y_pred))

            print()
            print("-"*80)
            print()

        except Exception as e:
            print(f"# ERRO: não foi possível carregar os dados do aparelho `{CARGA} - {INSTANCIA}` ({str(e)})")
            print()
    
# Exportando resultados
df_baseline = pd.DataFrame(resultados)
df_baseline["metodologia"] = "Baseline"
df_baseline.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_baseline.xlsx"),
    index=False
)

* Aparelho FRIDGE:

   - Carregando dados (taxa=8, janela=1080 - instancia 8)...
# ERRO: não foi possível carregar os dados do aparelho `fridge - 8` ('NoneType' object has no attribute 'values')

* Aparelho DISH_WASHER:

   - Carregando dados (taxa=5, janela=360 - instancia 9)...
# ERRO: não foi possível carregar os dados do aparelho `dish_washer - 9` ('NoneType' object has no attribute 'values')

* Aparelho WASHER_DRYER:

   - Carregando dados (taxa=3, janela=900 - instancia 4)...
# ERRO: não foi possível carregar os dados do aparelho `washer_dryer - 4` ('NoneType' object has no attribute 'values')

* Aparelho WASHER_DRYER:

   - Carregando dados (taxa=3, janela=900 - instancia 4)...
# ERRO: não foi possível carregar os dados do aparelho `washer_dryer - 4` ('NoneType' object has no attribute 'values')

* Aparelho MICROWAVE:



### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP
    
    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):
        
        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1

            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                ##################### METODOLOGIA ANTIGA #####################
                # Modelo baseline
                modelo = convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                )
                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando
                y_hat = modelo.predict(X_teste_imagem).round().astype(np.int16)
                ##################### METODOLOGIA ANTIGA #####################

                tf.keras.backend.clear_session()

                # Incrementando resultados
                acc = accuracy_score(y_teste, y_hat)
                score = f1_score(y_teste, y_hat, average="macro")
                try:
                    auc_score = roc_auc_score(y_teste, y_hat)
                except:
                    if score == 1:
                        auc_score = 1
                    else:
                        auc_score = 0.5
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste)
                y_pred.extend(y_hat)

                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao)
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_baseline_cv = pd.DataFrame(resultados)
df_baseline_cv["metodologia"] = "Baseline"
df_baseline_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_baseline_cv.xlsx"),
    index=False
)

## Metodologia Final/TCE (Transfer Learning, Class Weight e Early Stopping)

In [ ]:
metricas = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
]
early_stopping_auc = tf.keras.callbacks.EarlyStopping(
    min_delta=1e-5,
    monitor='val_auc', 
    verbose=0,
    patience=10,
    mode='max',
    restore_best_weights=True
)

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        ########################### METODOLOGIA FINAL ###########################
        # Selecionar informacoes sobre aparelhos em outras residencias
        dados_pretreino = {}
        for r in list(set([1,2,3,4,5,6]) - set([RESIDENCIA])):
            for e in redd.buildings[r].elec.meters:
                if e.label().lower().replace(" ","_") == CARGA:
                    dados_pretreino[r] = e.instance() if not hasattr(e,'meters') else e.instance()[0]
        modelo_pretreino = convnet_metodologia(
            metrics=metricas,
            optimizer=tf.keras.optimizers.Adam(1e-3), 
            #bias_output=bias_inicial_output
        )    
        # Realizando pre-treinamento
        print(f"   - Realizando pré-treinamento do modelo ({dados_pretreino})...")
        for res, inst in dados_pretreino.items():
            try:
                X_pretreino, y_pretreino = carregar_dados_aparelho(
                    janelas=Janelas(
                        base=redd,
                        id_residencia=res,
                        inicio_intervalo=PARAMETROS_DATASET["inicio_intervalo"],
                        fim_intervalo=PARAMETROS_DATASET["fim_intervalo"],
                        debug=False
                    ),
                    instancia=inst, 
                    aparelho=CARGA, 
                    taxa=TAXA,
                    tamanho_janela=TAMANHO_JANELA,
                )
                X_pretreino, y_pretreino = preparar_amostras(
                    X_pretreino, y_pretreino, 
                    params_rp=CONFIG_RP_APARELHO,
                    tam_imagem=TAMANHO_IMAGEM,
                    normalizar=False 
                ) 

                # Normalizar dados p/ modelo
                X_pretreino = normalizar_dados(X_pretreino)

                # calculando punicao para classes (desbalanceamento)
                try:
                    neg, pos = np.bincount(y_pretreino)
                except:
                    dist = Counter(y_pretreino)
                    neg, pos = dist[0], dist[1]
                    neg += 1e-5
                    pos += 1e-5
                    del dist
                total = neg + pos
                p0 = (1 / neg)*(total)/2.0 
                p1 = (1 / pos)*(total)/2.0
                pesos_classes_tl = {
                    0: p0 if not np.isinf(p0) else 1e-3, 
                    1: p1 if not np.isinf(p1) else 1e-3
                }

                # Treinando
                historico = modelo_pretreino.fit(
                    X_pretreino, y_pretreino,
                    validation_data=(X_pretreino, y_pretreino),
                    epochs=EPOCAS,
                    batch_size=int(len(y_pretreino)/3),#TAMANHO_LOTE_AUMENTADO,
                    class_weight=pesos_classes_tl,
                    callbacks=[early_stopping_auc],
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando
                y_hat = modelo_pretreino.predict(X_pretreino).round().astype(np.int16)

                print(classification_report(y_pretreino, y_hat))
                print()
                print(confusion_matrix(y_pretreino, y_hat))
                print()

                tf.keras.backend.clear_session()

                # Incrementando resultados
                score = f1_score(y_pretreino, y_hat, average="macro")
                auc_score = roc_auc_score(y_pretreino, y_hat) if np.unique(y_pretreino).shape[0]>1 else 1
                print("      -> Resultados residencia #{}: F1-score = {:.2f}% / AUC = {:.3f}".format(
                    res,
                    score*100,
                    auc_score,
                ))
                print()
            except Exception as e:
                print(f"      -> Resultados residencia #{res}: Não foi possível "+\
                      "pré-treinar o modelo com os dados desta residência.")
                print(f"         # MOTIVO: {str(e)}")
                print()


        # calculando punicao para classes (desbalanceamento)
        try:
            neg, pos = np.bincount(y_treino)
        except:
            dist = Counter(y_treino)
            neg, pos = dist[0], dist[1]
            neg += 1e-5
            pos += 1e-5
            del dist
        total = neg + pos
        p0 = (1 / neg)*(total)/2.0 
        p1 = (1 / pos)*(total)/2.0
        pesos_classes = {
            0: p0 if not np.isinf(p0) else 1e-3, 
            1: p1 if not np.isinf(p1) else 1e-3
        }
        print("   - Punição de classes:", pesos_classes)
        print()

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []
        suportes_relativos = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            # Instanciando modelo pre-treinado
            modelo = modelo_pretreino
            modelo.set_weights(modelo_pretreino.get_weights())

            # Treinando
            historico = modelo.fit(
                X_treino_imagem, y_treino,
                validation_data=(X_teste_imagem, y_teste),
                epochs=EPOCAS,
                batch_size=int(len(y_treino)/4),#TAMANHO_LOTE**2, # TODO: Validar heuristica do tamanho do lote
                class_weight=pesos_classes,
                callbacks=[early_stopping_auc],
                #verbose=VERBOSIDADE
                verbose=0
            )

            y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                classificacao_threshold(
                    modelo, X_teste_imagem, y_teste, 
                    threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                    grafico=False
                )

            tf.keras.backend.clear_session()

            # Incrementando resultados
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste_)
            y_pred.extend(y_pred_)

            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)


    print()
    print("   - Resultados finais:")
    print("   ---")
    print()


    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()

In [ ]:
# Exportando resultados
df_tce = pd.DataFrame(resultados)
df_tce["metodologia"] = "Fnal/TCE"
df_tce.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_tce.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            ########################### METODOLOGIA FINAL ###########################
            # Selecionar informacoes sobre aparelhos em outras residencias
            dados_pretreino = {}
            for r in list(set([1,2,3,4,5,6]) - set([RESIDENCIA])):
                for e in redd.buildings[r].elec.meters:
                    if e.label().lower().replace(" ","_") == CARGA:
                        dados_pretreino[r] = e.instance() if not hasattr(e,'meters') else e.instance()[0]
            modelo_pretreino = convnet_metodologia(
                metrics=metricas,
                optimizer=tf.keras.optimizers.Adam(1e-3), 
                #bias_output=bias_inicial_output
            )    
            # Realizando pre-treinamento
            print(f"   - Realizando pré-treinamento do modelo ({dados_pretreino})...")
            for res, inst in dados_pretreino.items():
                try:
                    X_pretreino, y_pretreino = carregar_dados_aparelho(
                        janelas=Janelas(
                            base=redd,
                            id_residencia=res,
                            inicio_intervalo=PARAMETROS_DATASET["inicio_intervalo"],
                            fim_intervalo=PARAMETROS_DATASET["fim_intervalo"],
                            debug=False
                        ),
                        instancia=inst, 
                        aparelho=CARGA, 
                        taxa=TAXA,
                        tamanho_janela=TAMANHO_JANELA,
                    )
                    X_pretreino, y_pretreino = preparar_amostras(
                        X_pretreino, y_pretreino, 
                        params_rp=CONFIG_RP_APARELHO,
                        tam_imagem=TAMANHO_IMAGEM,
                        normalizar=False 
                    ) 

                    # Normalizar dados p/ modelo
                    X_pretreino = normalizar_dados(X_pretreino)

                    # calculando punicao para classes (desbalanceamento)
                    try:
                        neg, pos = np.bincount(y_pretreino)
                    except:
                        dist = Counter(y_pretreino)
                        neg, pos = dist[0], dist[1]
                        neg += 1e-5
                        pos += 1e-5
                        del dist
                    total = neg + pos
                    p0 = (1 / neg)*(total)/2.0 
                    p1 = (1 / pos)*(total)/2.0
                    pesos_classes_tl = {
                        0: p0 if not np.isinf(p0) else 1e-3, 
                        1: p1 if not np.isinf(p1) else 1e-3
                    }

                    # Treinando
                    historico = modelo_pretreino.fit(
                        X_pretreino, y_pretreino,
                        validation_data=(X_pretreino, y_pretreino),
                        epochs=EPOCAS,
                        batch_size=int(len(y_pretreino)/3),#TAMANHO_LOTE_AUMENTADO,
                        class_weight=pesos_classes_tl,
                        callbacks=[early_stopping_auc],
                        #verbose=VERBOSIDADE
                        verbose=0
                    )

                    # Avaliando
                    y_hat = modelo_pretreino.predict(X_pretreino).round().astype(np.int16)

                    print(classification_report(y_pretreino, y_hat))
                    print()
                    print(confusion_matrix(y_pretreino, y_hat))
                    print()

                    tf.keras.backend.clear_session()

                    # Incrementando resultados
                    score = f1_score(y_pretreino, y_hat, average="macro")
                    auc_score = roc_auc_score(y_pretreino, y_hat) if np.unique(y_pretreino).shape[0]>1 else 1
                    print("      -> Resultados residencia #{}: F1-score = {:.2f}% / AUC = {:.3f}".format(
                        res,
                        score*100,
                        auc_score,
                    ))
                    print()
                except Exception as e:
                    print(f"      -> Resultados residencia #{res}: Não foi possível "+\
                          "pré-treinar o modelo com os dados desta residência.")
                    print(f"         # MOTIVO: {str(e)}")
                    print()


            # calculando punicao para classes (desbalanceamento)
            try:
                neg, pos = np.bincount(y_treino)
            except:
                dist = Counter(y_treino)
                neg, pos = dist[0], dist[1]
                neg += 1e-5
                pos += 1e-5
                del dist
            total = neg + pos
            p0 = (1 / neg)*(total)/2.0 
            p1 = (1 / pos)*(total)/2.0
            pesos_classes = {
                0: p0 if not np.isinf(p0) else 1e-3, 
                1: p1 if not np.isinf(p1) else 1e-3
            }
            print("   - Punição de classes:", pesos_classes)
            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []
            suportes_relativos = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                # Instanciando modelo pre-treinado
                modelo = modelo_pretreino
                modelo.set_weights(modelo_pretreino.get_weights())

                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=int(len(y_treino)/4),#TAMANHO_LOTE**2, # TODO: Validar heuristica do tamanho do lote
                    class_weight=pesos_classes,
                    callbacks=[early_stopping_auc],
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                    classificacao_threshold(
                        modelo, X_teste_imagem, y_teste, 
                        threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                        grafico=False
                    )


                tf.keras.backend.clear_session()

                # Incrementando resultados
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste_)
                y_pred.extend(y_pred_)


                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)
                
    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_tce_cv = pd.DataFrame(resultados)
df_tce_cv["metodologia"] = "Fnal/TCE"
df_tce_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_tce_cv.xlsx"),
    index=False
)

**NOTA:** O desbalanceamento pode estar impactar MUITO a convergência do modelo. Todavia, é esperado esta característica nos dados de NILM.

# Análise dos Resultados (1)

## Split Treino/Teste

In [ ]:
df_baseline = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_baseline.xlsx"))
df_tce = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_tce.xlsx"))

df_analise = pd.concat([
    # Resultados das metodologias Baseline e Final
    df_baseline, 
    df_tce,
  
])
print("* Análise por metodologia:")
df_analise_metodologia = df_analise.groupby(["metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("metodologia")
display(df_analise_metodologia)
df_analise_metodologia.to_excel(os.path.join(caminho_dados_notebook, "df_analise1_metodologia.xlsx"))

print()
print("* Análise por aparelho/metodologia:")
df_analise_aparelho = df_analise.groupby(["aparelho","metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise1_aparelho.xlsx"))

## Validação Cruzada

In [ ]:
df_baseline_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_baseline_cv.xlsx"))
df_tce_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_tce_cv.xlsx"))

df_analise_cv = pd.concat([
    # Resultados das metodologias Baseline e Final
    df_baseline_cv, 
    df_tce_cv,  
])

print("* Análise por metodologia:")
df_analise_metodologia = df_analise_cv.groupby(["metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("metodologia")
display(df_analise_metodologia)
df_analise_metodologia.to_excel(os.path.join(caminho_dados_notebook, "df_analise1_metodologia_cv.xlsx"))

print()
print("* Análise por aparelho/metodologia:")
df_analise_aparelho = df_analise_cv.groupby(["aparelho","metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise1_aparelho_cv.xlsx"))

**CONCLUSÕES:**

1. A Metodologia Final/TCE melhorou apenas o aparelho Microwave (+9%), tendo desempenho PIOR em todos os outros
2. Analisando os dados do aparelho Fridge, é possível notar um fenômeno interessante: as amostras negativas refletem a momentos que TODOS os aparelhos estão desligados (fica a questão: é útil considerar este aparelho nas análises?);

Em resumo, percebe-se que a metodologia final/TCE é instável. Este comportamento pode estar AINDA diretamente envolvido com o FORTE desbalanceamento das classes, de modo que o Transfer Learning em outras residências (que também possuem poucas amostras) não fornece conhecimento necessário para o modelo. 

Vale ressaltar que as classes raras (< 5% das amostras) apresentam um grande desafio para modelos baseados em retropropagação, pois o cálculo do gradientes para os erros nessas amostras serão irrelevantes no processo de treinamento.

Sendo assim, é válido avaliar estratégias que de alguma forna EQUILIBRE a disponibilização de amostras raras no cálculo do gradiente. Logo, vamos desenvolver uma nova metodologia, baseada na geração equilibrada de classes nos minilotes. Dado que o aparelho que possui MAIS AMOSTRAS da classe rara no conjunto de treino (Microwave) contempla apenas 16 amostras desta, assumiremos que preservar estas durante toda a época, selecionando apenas a mesma quantidade de amostras MAJORITÁRIAS em cada minilote.

Chamaremos essa metodologia de Treinamento com Minilotes Equilibrados, baseado nos trabalhos de [Shimizu et. al (2020)](https://ieeexplore.ieee.org/document/8665709) e [Cappelletti et. al](https://www.sensorsportal.com/HTML/DIGEST/P_3087.htm).

# Hipóteses para Lidar com Desbalanceamento

## Metodologia: Transfer Learning (Baseline)

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        ########################### METODOLOGIA FINAL ###########################
        # Selecionar informacoes sobre aparelhos em outras residencias
        dados_pretreino = {}
        for r in list(set([1,2,3,4,5,6]) - set([RESIDENCIA])):
            for e in redd.buildings[r].elec.meters:
                if e.label().lower().replace(" ","_") == CARGA:
                    dados_pretreino[r] = e.instance() if not hasattr(e,'meters') else e.instance()[0]
        modelo_pretreino = convnet_metodologia(
            metrics=metricas,
            optimizer=tf.keras.optimizers.Adam(1e-3), 
            #bias_output=bias_inicial_output
        )    
        # Realizando pre-treinamento
        print(f"   - Realizando pré-treinamento do modelo ({dados_pretreino})...")
        for res, inst in dados_pretreino.items():
            try:
                X_pretreino, y_pretreino = carregar_dados_aparelho(
                    janelas=Janelas(
                        base=redd,
                        id_residencia=res,
                        inicio_intervalo=PARAMETROS_DATASET["inicio_intervalo"],
                        fim_intervalo=PARAMETROS_DATASET["fim_intervalo"],
                        debug=False
                    ),
                    instancia=inst, 
                    aparelho=CARGA, 
                    taxa=TAXA,
                    tamanho_janela=TAMANHO_JANELA,
                )
                X_pretreino, y_pretreino = preparar_amostras(
                    X_pretreino, y_pretreino, 
                    params_rp=CONFIG_RP_APARELHO,
                    tam_imagem=TAMANHO_IMAGEM,
                    normalizar=False 
                ) 

                # Normalizar dados p/ modelo
                X_pretreino = normalizar_dados(X_pretreino)

                # calculando punicao para classes (desbalanceamento)
                try:
                    neg, pos = np.bincount(y_pretreino)
                except:
                    dist = Counter(y_pretreino)
                    neg, pos = dist[0], dist[1]
                    neg += 1e-5
                    pos += 1e-5
                    del dist
                p0 = (1 / neg)*(total)/2.0 
                p1 = (1 / pos)*(total)/2.0
                pesos_classes_tl = {
                    0: p0 if not np.isinf(p0) else 1e-3, 
                    1: p1 if not np.isinf(p1) else 1e-3
                }

                # Treinando
                historico = modelo_pretreino.fit(
                    X_pretreino, y_pretreino,
                    validation_data=(X_pretreino, y_pretreino),
                    epochs=EPOCAS,
                    batch_size=int(len(y_pretreino)/3),#TAMANHO_LOTE_AUMENTADO,
                    class_weight=pesos_classes_tl,
                    callbacks=[early_stopping_auc],
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando
                y_hat = modelo_pretreino.predict(X_pretreino).round().astype(np.int16)

                print(classification_report(y_pretreino, y_hat))
                print()
                print(confusion_matrix(y_pretreino, y_hat))
                print()

                tf.keras.backend.clear_session()

                # Incrementando resultados
                score = f1_score(y_pretreino, y_hat, average="macro")
                auc_score = roc_auc_score(y_pretreino, y_hat) if np.unique(y_pretreino).shape[0]>1 else 1
                print("      -> Resultados residencia #{}: F1-score = {:.2f}% / AUC = {:.3f}".format(
                    res,
                    score*100,
                    auc_score,
                ))
                print()
            except Exception as e:
                print(f"      -> Resultados residencia #{res}: Não foi possível "+\
                      "pré-treinar o modelo com os dados desta residência.")
                print(f"         # MOTIVO: {str(e)}")
                print()


        # calculando punicao para classes (desbalanceamento)
        try:
            neg, pos = np.bincount(y_treino)
        except:
            dist = Counter(y_treino)
            neg, pos = dist[0], dist[1]
            neg += 1e-5
            pos += 1e-5
            del dist
        total = neg + pos
        p0 = (1 / neg)*(total)/2.0 
        p1 = (1 / pos)*(total)/2.0
        pesos_classes = {
            0: p0 if not np.isinf(p0) else 1e-3, 
            1: p1 if not np.isinf(p1) else 1e-3
        }
        print("   - Punição de classes:", pesos_classes)
        print()

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []
        suportes_relativos = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            # Instanciando modelo pre-treinado
            modelo = modelo_pretreino
            modelo.set_weights(modelo_pretreino.get_weights())

            # Treinando
            historico = modelo.fit(
                X_treino_imagem, y_treino,
                validation_data=(X_teste_imagem, y_teste),
                epochs=EPOCAS,
                batch_size=TAMANHO_LOTE,
                #verbose=VERBOSIDADE
                verbose=0
            )

            y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                classificacao_threshold(
                    modelo, X_teste_imagem, y_teste, 
                    threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                    grafico=False
                )


            tf.keras.backend.clear_session()

            # Incrementando resultados
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste_)
            y_pred.extend(y_pred_)


            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_transfer_learning = pd.DataFrame(resultados)
df_transfer_learning["metodologia"] = "Transfer Learning (Baseline)"
df_transfer_learning.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_transfer_learning.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
   # "suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()


            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            ########################### METODOLOGIA FINAL ###########################
            # Selecionar informacoes sobre aparelhos em outras residencias
            dados_pretreino = {}
            for r in list(set([1,2,3,4,5,6]) - set([RESIDENCIA])):
                for e in redd.buildings[r].elec.meters:
                    if e.label().lower().replace(" ","_") == CARGA:
                        dados_pretreino[r] = e.instance() if not hasattr(e,'meters') else e.instance()[0]
            modelo_pretreino = convnet_metodologia(
                metrics=metricas,
                optimizer=tf.keras.optimizers.Adam(1e-3), 
                #bias_output=bias_inicial_output
            )    
            # Realizando pre-treinamento
            print(f"   - Realizando pré-treinamento do modelo ({dados_pretreino})...")
            for res, inst in dados_pretreino.items():
                try:
                    X_pretreino, y_pretreino = carregar_dados_aparelho(
                        janelas=Janelas(
                            base=redd,
                            id_residencia=res,
                            inicio_intervalo=PARAMETROS_DATASET["inicio_intervalo"],
                            fim_intervalo=PARAMETROS_DATASET["fim_intervalo"],
                            debug=False
                        ),
                        instancia=inst, 
                        aparelho=CARGA, 
                        taxa=TAXA,
                        tamanho_janela=TAMANHO_JANELA,
                    )
                    X_pretreino, y_pretreino = preparar_amostras(
                        X_pretreino, y_pretreino, 
                        params_rp=CONFIG_RP_APARELHO,
                        tam_imagem=TAMANHO_IMAGEM,
                        normalizar=False 
                    ) 

                    # Normalizar dados p/ modelo
                    X_pretreino = normalizar_dados(X_pretreino)

                    # calculando punicao para classes (desbalanceamento)
                    try:
                        neg, pos = np.bincount(y_pretreino)
                    except:
                        dist = Counter(y_pretreino)
                        neg, pos = dist[0], dist[1]
                        neg += 1e-5
                        pos += 1e-5
                        del dist
                    total = neg + pos
                    p0 = (1 / neg)*(total)/2.0 
                    p1 = (1 / pos)*(total)/2.0
                    pesos_classes_tl = {
                        0: p0 if not np.isinf(p0) else 1e-3, 
                        1: p1 if not np.isinf(p1) else 1e-3
                    }

                    # Treinando
                    historico = modelo_pretreino.fit(
                        X_pretreino, y_pretreino,
                        validation_data=(X_pretreino, y_pretreino),
                        epochs=EPOCAS,
                        batch_size=int(len(y_pretreino)/3),#TAMANHO_LOTE_AUMENTADO,
                        class_weight=pesos_classes_tl,
                        callbacks=[early_stopping_auc],
                        #verbose=VERBOSIDADE
                        verbose=0
                    )
                    
                    # Avaliando
                    y_hat = modelo_pretreino.predict(X_pretreino).round().astype(np.int16)

                    print(classification_report(y_pretreino, y_hat))
                    print()
                    print(confusion_matrix(y_pretreino, y_hat))
                    print()

                    tf.keras.backend.clear_session()

                    # Incrementando resultados
                    score = f1_score(y_pretreino, y_hat, average="macro")
                    auc_score = roc_auc_score(y_pretreino, y_hat) if np.unique(y_pretreino).shape[0]>1 else 1
                    print("      -> Resultados residencia #{}: F1-score = {:.2f}% / AUC = {:.3f}".format(
                        res,
                        score*100,
                        auc_score,
                    ))
                    print()
                except Exception as e:
                    print(f"      -> Resultados residencia #{res}: Não foi possível "+\
                          "pré-treinar o modelo com os dados desta residência.")
                    print(f"         # MOTIVO: {str(e)}")
                    print()


            # calculando punicao para classes (desbalanceamento)
            try:
                neg, pos = np.bincount(y_treino)
            except:
                dist = Counter(y_treino)
                neg, pos = dist[0], dist[1]
                neg += 1e-5
                pos += 1e-5
                del dist
            total = neg + pos
            p0 = (1 / neg)*(total)/2.0 
            p1 = (1 / pos)*(total)/2.0
            pesos_classes = {
                0: p0 if not np.isinf(p0) else 1e-3, 
                1: p1 if not np.isinf(p1) else 1e-3
            }
            print("   - Punição de classes:", pesos_classes)
            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []
            suportes_relativos = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                # Instanciando modelo pre-treinado
                modelo = modelo_pretreino
                modelo.set_weights(modelo_pretreino.get_weights())

                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    #verbose=VERBOSIDADE
                    verbose=0
                )


                y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                    classificacao_threshold(
                        modelo, X_teste_imagem, y_teste, 
                        threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                        grafico=False
                    )


                tf.keras.backend.clear_session()

                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste_)
                y_pred.extend(y_pred_)


                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_transfer_learning_cv = pd.DataFrame(resultados)
df_transfer_learning_cv["metodologia"] = "Transfer Learning (Baseline)"
df_transfer_learning_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_transfer_learning_cv.xlsx"),
    index=False
)

## Metodologia: Treinamento Estratégico (Minilotes Equilibrados)

In [ ]:
# Rotina de treinamento estratificado (ideia principal: usar o lote de positivos
# em todos os minibatchs de treinamento do modelo)
def treinamento_estrategico(
    modelo,
    X, y,
    validacao=None,
    epocas=5,
    majoritaria=0,
    minoritaria=1,
    peso_classe='auto',
    debug = True
):
    from sklearn.utils import shuffle
    
    if validacao:
        X_teste = validacao[0]
        y_teste = validacao[1]
    
    # Totalizando por classe
    total = Counter(y)
    total_maj = total[majoritaria]
    total_min = total[minoritaria]
    
    if peso_classe == "auto":
        peso_classe = {
            majoritaria: 1,
            minoritaria: total_maj/total_min
        }

    # Selecionando indice das amostras por classe
    idx_min = [y_[0] for y_ in np.argwhere(y == minoritaria)]
    idx_maj = [y_[0] for y_ in np.argwhere(y == majoritaria)]

    for e in range(epocas):

        if debug: print(f"Época {e+1}")

        np.random.shuffle(idx_maj)
        np.random.shuffle(idx_min)
        for b,i in enumerate(np.arange(total_maj, step=total_min)):

            # Lote da classe maioritaria
            idx_lote = idx_maj[i:i+total_min]

            # Lote final (shuffle majoritaria + minoritaria)
            X_lote, y_lote = shuffle(
                np.concatenate([X[idx_lote], X[idx_min]]),
                np.concatenate([y[idx_lote], y[idx_min]]),
            )
            # Treinando no lote equilibrado
            m_treino = modelo.train_on_batch(X_lote, y_lote, class_weight=peso_classe)

            if debug:
                if validacao is None:
                    sys.stdout.write('\r - Batch {}/{}: {}'.format(
                        b, int(np.floor(total_maj/total_min)),
                        " ".join([f"{metrica}: {round(valor,4) if metrica=='loss' else round(valor,2)}" for metrica, valor in dict(zip(modelo.metrics_names, m_treino)).items()]))
                    )
                    sys.stdout.flush()
                else:
                    m_teste = modelo.evaluate(X_teste, verbose=0)
                    sys.stdout.write('\r - Batch {}/{}: {} | {}'.format(
                        b, int(np.floor(total_maj/total_min)),
                        " ".join([f"{metrica}: {round(valor,4) if metrica=='loss' else round(valor,2)}" for metrica, valor in dict(zip(modelo.metrics_names, m_treino)).items()]),
                        " ".join([f"val_{metrica}: {round(valor,4) if metrica=='loss' else round(valor,3)}" for metrica, valor in dict(zip(modelo.metrics_names, m_teste)).items()]))
                    )
                    sys.stdout.flush()
        if debug: print()
    
    if debug: print()
    return modelo

# Rotina de construcao DINAMICA do modelo (baseado em hiperparametrizacao)
def build(
    optimizer='adam',
    loss = 'binary_crossentropy', 
    metrics=['accuracy'],
    bias_output = None,
    arquitetura={
        "conv": {
            "modules":1, 
            "layers": 1,
            "filters": 8,
            "kernel_size":3,
            "strides":1,
            "padding":"same"
        },
        "pooling": {
            "mode": "max",
            "size":2,
            "strides": 1,
            "padding": "same"
        }
    },
    initializer=None
):
        
    model = Sequential()
    
    model.add(tf.keras.layers.Input(shape=(32,32,1)))
    
    conv = arquitetura["conv"]
    pooling = arquitetura["pooling"]
    
    for _ in range(conv["modules"]):
        for _ in range(conv["layers"]):
                                  
            if initializer is not None:
                init = initializer
            else:
                #https://www.researchgate.net/post/Proper_Weight_Initialization_for_ReLU_and_PReLU
                init = tf.keras.initializers.RandomNormal(
                    stddev=np.sqrt(1/(conv["kernel_size"]*conv["kernel_size"]*1))
                ) 
                                          
            model.add(Conv2D(
                filters=conv["filters"], 
                kernel_size=conv["kernel_size"], 
                strides=conv["strides"], 
                padding=conv["padding"], 
                activation="relu",
                #kernel_initializer=initializer
                kernel_initializer=init
            ))
        if pooling["mode"] == "max":
            model.add(MaxPooling2D(
                pool_size=pooling["size"], 
                strides=pooling["strides"], 
                padding=pooling["padding"], 
            ))    
        else:
            model.add(AveragePooling2D(
                pool_size=pooling["size"], 
                strides=pooling["strides"], 
                padding=pooling["padding"], 
            ))   
            
    model.add(Flatten())
                                  
    model.add(Dense(128, activation='relu', 
                    #kernel_initializer=tf.keras.initializers.RandomNormal(stddev=np.sqrt(2/128*128))
                   ))
#     model.add(Dropout(0.25))
    
    if bias_output is not None:
        bias_output = tf.keras.initializers.Constant(bias_output)

    model.add(Dense(1, activation='sigmoid', bias_initializer=bias_output))

    model.compile(optimizer=optimizer, loss=[loss], metrics=metrics)
    
    return model
                                  
# # Instanciando o modelo
# modelo = build(
#     optimizer=tf.keras.optimizers.Adam(1e-3),
#     #loss_function='binary_crossentropy',#focal_loss.BinaryFocalLoss(gamma=0.1),        
#     loss='binary_crossentropy',
#     metrics=metricas,
# )
# modelo.summary()
def classe_majoritaria(d):
    """ a) criar lista de chaves e valores; 
     b) return da chave com mais ocorrencias"""  
    v=list(d.values())
    k=list(d.keys())
    return k[v.index(max(v))]
def classe_minoritaria(d):
    """ a) criar lista de chaves e valores; 
     b) return da chave com menos ocorrencias"""  
    v=list(d.values())
    k=list(d.keys())
    return k[v.index(min(v))]

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP


    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            ##################### METODOLOGIA ANTIGA #####################
            # Treinamento estratico (lotes equilibrados)
            modelo = treinamento_estrategico(
                modelo=convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                ), 
                X=X_treino_imagem, y=y_treino, 
                #validacao=(X_teste_imagem, y_teste), 
                epocas=EPOCAS, 
                majoritaria=classe_majoritaria(Counter(y_treino)), 
                minoritaria=classe_minoritaria(Counter(y_treino)), 
                peso_classe={classe_majoritaria(Counter(y_treino)):1, 
                             classe_minoritaria(Counter(y_treino)):1}, 
                debug=False
            )

            # Avaliando
            y_hat = modelo.predict(X_teste_imagem).round().astype(np.int16)
            ##################### METODOLOGIA ANTIGA #####################

            tf.keras.backend.clear_session()

            # Incrementando resultados
            acc = accuracy_score(y_teste, y_hat)
            score = f1_score(y_teste, y_hat, average="macro")
            try:
                auc_score = roc_auc_score(y_teste, y_hat)
            except:
                if score == 1:
                    auc_score = 1
                else:
                    auc_score = 0.5
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste)
            y_pred.extend(y_hat)

            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_treinamento_estrategico = pd.DataFrame(resultados)
df_treinamento_estrategico["metodologia"] = "Treinamento Estratégico"
df_treinamento_estrategico.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_treinamento_estrategico.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        print("   - Detalhes da amostragem (lotes):")
        print("   ---")
        for item in Counter(y).items():
            print(f"      - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
        print()

        # Checando series estaveis
        estavel = []
        for i, x in enumerate(X):
            if len(np.unique(x)) == 1:
                estavel.append(i)
        print("      - Séries estáveis (1 amplitude)         : {} ({:.2f}%)".format(len(estavel), len(estavel)/len(X)*100) )
        print("      - Distribuicao de classes nestas séries :", Counter(y[estavel]))
        print()

        print("      - Estatísticas das séries: = Min. / Max. / Média / STD:", X.min(), X.max(), X.mean(), X.std())
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                ##################### METODOLOGIA ANTIGA #####################
                # Treinamento estratico (lotes equilibrados)
                modelo = treinamento_estrategico(
                    modelo=convnet_metodologia(
                        input_shape_= TAMANHO_IMAGEM,
                        output_dim = 1,
                        loss_function='binary_crossentropy',
                        metrics=['accuracy'],
                        output_activation='sigmoid'
                    ), 
                    X=X_treino_imagem, y=y_treino, 
                    #validacao=(X_teste_imagem, y_teste), 
                    epocas=EPOCAS, 
                    majoritaria=classe_majoritaria(Counter(y_treino)), 
                    minoritaria=classe_minoritaria(Counter(y_treino)), 
                    peso_classe={classe_majoritaria(Counter(y_treino)):1, 
                                 classe_minoritaria(Counter(y_treino)):1}, 
                    debug=False
                )

                # Avaliando
                y_hat = modelo.predict(X_teste_imagem).round().astype(np.int16)
                ##################### METODOLOGIA ANTIGA #####################

                tf.keras.backend.clear_session()

                # Incrementando resultados
                acc = accuracy_score(y_teste, y_hat)
                score = f1_score(y_teste, y_hat, average="macro")
                try:
                    auc_score = roc_auc_score(y_teste, y_hat)
                except:
                    if score == 1:
                        auc_score = 1
                    else:
                        auc_score = 0.5
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste)
                y_pred.extend(y_hat)

                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
# Exportando resultados
df_treinamento_estrategico_cv = pd.DataFrame(resultados)
df_treinamento_estrategico_cv["metodologia"] = "Treinamento Estratégico"
df_treinamento_estrategico_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_treinamento_estrategico_cv.xlsx"),
    index=False
)

## Metodologia: Class Weight

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        # calculando punicao para classes (desbalanceamento)
        try:
            neg, pos = np.bincount(y_treino)
        except:
            dist = Counter(y_treino)
            neg, pos = dist[0], dist[1]
            neg += 1e-5
            pos += 1e-5
            del dist
        total = neg + pos
        p0 = (1 / neg)*(total)/2.0 
        p1 = (1 / pos)*(total)/2.0
        pesos_classes = {
            0: p0 if not np.isinf(p0) else 1e-3, 
            1: p1 if not np.isinf(p1) else 1e-3
        }
        print("   - Punição de classes:", pesos_classes)
        print()

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []
        suportes_relativos = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            # Instanciando modelo pre-treinado
            modelo = convnet_metodologia(
                input_shape_= TAMANHO_IMAGEM,
                output_dim = 1,
                loss_function='binary_crossentropy',
                metrics=['accuracy'],
                output_activation='sigmoid'
            )

            # Treinando
            historico = modelo.fit(
                X_treino_imagem, y_treino,
                validation_data=(X_teste_imagem, y_teste),
                epochs=EPOCAS,
                batch_size=TAMANHO_LOTE,
                class_weight=pesos_classes,
                #verbose=VERBOSIDADE
                verbose=0
            )

            y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                classificacao_threshold(
                    modelo, X_teste_imagem, y_teste, 
                    threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                    grafico=False
                )


            tf.keras.backend.clear_session()

            # Incrementando resultados
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste_)
            y_pred.extend(y_pred_)


            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)


    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_classweight = pd.DataFrame(resultados)
df_classweight["metodologia"] = "Class Weight"
df_classweight.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_classweight.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            # calculando punicao para classes (desbalanceamento)
            try:
                neg, pos = np.bincount(y_treino)
            except:
                dist = Counter(y_treino)
                neg, pos = dist[0], dist[1]
                neg += 1e-5
                pos += 1e-5
                del dist
            total = neg + pos
            p0 = (1 / neg)*(total)/2.0 
            p1 = (1 / pos)*(total)/2.0
            pesos_classes = {
                0: p0 if not np.isinf(p0) else 1e-3, 
                1: p1 if not np.isinf(p1) else 1e-3
            }
            print("   - Punição de classes:", pesos_classes)
            print()

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []
            suportes_relativos = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                # Instanciando modelo pre-treinado
                modelo = convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                )

                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    class_weight=pesos_classes,
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                    classificacao_threshold(
                        modelo, X_teste_imagem, y_teste, 
                        threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                        grafico=False
                    )

                tf.keras.backend.clear_session()

                # Incrementando resultados
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste_)
                y_pred.extend(y_pred_)


                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_classweight_cv = pd.DataFrame(resultados)
df_classweight_cv["metodologia"] = "Class Weight"
df_classweight_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_classweight_cv.xlsx"),
    index=False
)

## Metodologia: Random Over Sampling (ROS)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        ########################### OVERSAMPLING ###########################
        try:
            X_treino_imagem, y_treino = RandomOverSampler(random_state=SEED).fit_resample(
                X_treino_imagem.reshape(-1, np.product(TAMANHO_IMAGEM)), 
                y_treino
            )
            X_treino_imagem = X_treino_imagem.reshape(-1, *TAMANHO_IMAGEM)

            print("      - Nova Distribuição lote TREINO (Reamostragem):")
            print("        -> Treino:", Counter(y_treino))
            print()
        except Exception as e:
            print("# ERRO:", str(e))

        ########################### OVERSAMPLING ###########################

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []
        suportes_relativos = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            # Instanciando modelo pre-treinado
            modelo = convnet_metodologia(
                input_shape_= TAMANHO_IMAGEM,
                output_dim = 1,
                loss_function='binary_crossentropy',
                metrics=['accuracy'],
                output_activation='sigmoid'
            )

            # Treinando
            historico = modelo.fit(
                X_treino_imagem, y_treino,
                validation_data=(X_teste_imagem, y_teste),
                epochs=EPOCAS,
                batch_size=TAMANHO_LOTE,
                #verbose=VERBOSIDADE
                verbose=0
            )

            # Avaliando resultados
            y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                classificacao_threshold(
                    modelo, X_teste_imagem, y_teste, 
                    threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                    grafico=False
                )


            tf.keras.backend.clear_session()

            # Incrementando resultados
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste_)
            y_pred.extend(y_pred_)


            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_ros = pd.DataFrame(resultados)
df_ros["metodologia"] = "ROS (Baseline + RandomOverSampling)"
df_ros.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_ros.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            ########################### OVERSAMPLING ###########################
            try:
                X_treino_imagem, y_treino = RandomOverSampler(random_state=SEED).fit_resample(
                    X_treino_imagem.reshape(-1, np.product(TAMANHO_IMAGEM)), 
                    y_treino
                )
                X_treino_imagem = X_treino_imagem.reshape(-1, *TAMANHO_IMAGEM)

                print("      - Nova Distribuição lote TREINO (Reamostragem):")
                print("        -> Treino:", Counter(y_treino))
                print()

            except Exception as e:
                print("# ERRO:", str(e))
            ########################### OVERSAMPLING ###########################

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []
            suportes_relativos = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                # Instanciando modelo pre-treinado
                modelo = convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                )

                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando resultados
                y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                    classificacao_threshold(
                        modelo, X_teste_imagem, y_teste, 
                        threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                        grafico=False
                    )


                tf.keras.backend.clear_session()

                # Incrementando resultados
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste_)
                y_pred.extend(y_pred_)


                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()
    
    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_ros_cv = pd.DataFrame(resultados)
df_ros_cv["metodologia"] = "ROS (Baseline + RandomOverSampling)"
df_ros_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_ros_cv.xlsx"),
    index=False
)

## Metodologia: Random Under Sampling (RUS)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

### Split Treino/Teste

In [ ]:
resultados = {
    "aparelho": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem
        print("   - Preparando dados para modelagem (treino/teste)...")
        print("   ---")
        # Fazendo split dos dados (treino/teste)
        X_treino, X_teste, y_treino, y_teste = train_test_split(
            X, y, 
            test_size=FRACAO_TESTE,
            stratify=y if Counter(y)[0] > 1 and Counter(y)[1] > 1 else None,
            random_state=SEED
        )
        print()

        print("      - Distribuições dos lotes:")
        print("        -> Treino:", Counter(y_treino))
        print("        -> Teste :", Counter(y_teste))

        X_treino_imagem, _ = preparar_amostras(
            X_treino, y_treino, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )
        X_teste_imagem, _ = preparar_amostras(
            X_teste, y_teste, 
            params_rp=CONFIG_RP_APARELHO,
            tam_imagem=TAMANHO_IMAGEM,
            normalizar=False # config. estudo 17 = False
        )

        # Normalizar dados p/ modelo
        X_treino_imagem = normalizar_dados(X_treino_imagem)
        X_teste_imagem = normalizar_dados(X_teste_imagem)

        print()

        ########################### UNDERSAMPLING ###########################
        try:
            X_treino_imagem, y_treino = RandomUnderSampler(random_state=SEED).fit_resample(
                X_treino_imagem.reshape(-1, np.product(TAMANHO_IMAGEM)), 
                y_treino
            )
            X_treino_imagem = X_treino_imagem.reshape(-1, *TAMANHO_IMAGEM)

            print("      - Nova Distribuição lote TREINO (Reamostragem):")
            print("        -> Treino:", Counter(y_treino))
            print()

        except Exception as e:
            print("# ERRO:", str(e))
        ########################### UNDERSAMPLING ###########################

        print("   - Avaliando modelo, aguarde...")
        print("   ---")
        y_true, y_pred = [], []
        accs = []
        scores = []
        aucs = []
        suportes_relativos = []

        # Avaliando N vezes o conjunto treino/teste (consitencia)
        N = 10
        for i in tqdm_notebook(range(N)):

            # Instanciando modelo pre-treinado
            modelo = convnet_metodologia(
                input_shape_= TAMANHO_IMAGEM,
                output_dim = 1,
                loss_function='binary_crossentropy',
                metrics=['accuracy'],
                output_activation='sigmoid'
            )

            # Treinando
            historico = modelo.fit(
                X_treino_imagem, y_treino,
                validation_data=(X_teste_imagem, y_teste),
                epochs=EPOCAS,
                batch_size=TAMANHO_LOTE,
                #verbose=VERBOSIDADE
                verbose=0
            )

            # Avaliando resultados
            y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                classificacao_threshold(
                    modelo, X_teste_imagem, y_teste, 
                    threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                    grafico=False
                )


            tf.keras.backend.clear_session()

            # Incrementando resultados
            print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                i+1, 
                acc*100,
                score*100,
                auc_score,
            ))
            accs.append(acc)
            scores.append(score)
            aucs.append(auc_score)
            y_true.extend(y_teste_)
            y_pred.extend(y_pred_)

            # Guardando resultados do modelo
            resultados["aparelho"].append(rotulo_aparelho)
            #resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
            resultados["teste"].append(i+1)
            resultados["acuracia"].append(acc)
            resultados["f1"].append(score)
            resultados["auc"].append(auc_score)


    print()
    print("   - Resultados finais:")
    print("   ---")
    print()

    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_rus = pd.DataFrame(resultados)
df_rus["metodologia"] = "RUS (Baseline + RandomUnderSampling)"
df_rus.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_rus.xlsx"),
    index=False
)

### Validação Cruzada

In [ ]:
resultados = {
    "aparelho": [],
    "iteracao": [],
    "teste": [],
    "acuracia": [],
    "f1": [],
    #"suporte_relativo": [],
    "auc":[],
}
for rotulo_aparelho in df_melhores_taxas_janelas.loc[
    df_melhores_taxas_janelas["carga"].isin(
        ['dish_washer - 9','fridge - 7','microwave - 16','washer_dryer - 13',
         'washer_dryer - 14']),
    : ]["carga"].values:
    
    print(f"* Aparelho {rotulo_aparelho.upper()}:")
    print()
    
    # Informacoes da carga selecionada
    CARGA = rotulo_aparelho.split(" - ")[0]
    #INSTANCIA = int(rotulo_aparelho.split(" - ")[1])

    config_aparelho = df_melhores_taxas_janelas[
        df_melhores_taxas_janelas["carga"]==rotulo_aparelho
    ].to_dict("records")[0]
    TAXA = config_aparelho["taxa_amostragem"]
    TAMANHO_JANELA = config_aparelho["janela"]
    CONFIG_RP_APARELHO = PARAMETROS_RP

    # Percorrer instancias do aparelho na residencia
    for INSTANCIA in instancia_aparelho_residencia(CARGA, RESIDENCIA, base = redd):

        # Extrair series divididas em janelas para cada medidor
        print("   - Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
            TAXA, TAMANHO_JANELA
        ))
        X, y = carregar_dados_aparelho(
            janelas=janelas,
            instancia=INSTANCIA,
            aparelho=CARGA,
            tamanho_janela=TAMANHO_JANELA,
            taxa=TAXA,
            eliminar_janelas_vazias=True
        )
        print()

        # Convertendo series para imagem (CV)
        for it, (idx_treino, idx_teste) in enumerate(kfold.split(X, y)):

            iteracao = it + 1    
            print(f"   - Preparando dados para modelagem (cv-{iteracao})...")
            print("   ---")
            X_treino, X_teste = X[idx_treino], X[idx_teste]
            y_treino, y_teste = y[idx_treino], y[idx_teste]
            print()

            print("      - Distribuições dos lotes:")
            print("        -> Treino:", Counter(y_treino))
            print("        -> Teste :", Counter(y_teste))

            X_treino_imagem, _ = preparar_amostras(
                X_treino, y_treino, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )
            X_teste_imagem, _ = preparar_amostras(
                X_teste, y_teste, 
                params_rp=CONFIG_RP_APARELHO,
                tam_imagem=TAMANHO_IMAGEM,
                normalizar=False # config. estudo 17 = False
            )

            # Normalizar dados p/ modelo
            X_treino_imagem = normalizar_dados(X_treino_imagem)
            X_teste_imagem = normalizar_dados(X_teste_imagem)

            print()

            ########################### UNDERSAMPLING ###########################
            try:
                X_treino_imagem, y_treino = RandomUnderSampler(random_state=SEED).fit_resample(
                    X_treino_imagem.reshape(-1, np.product(TAMANHO_IMAGEM)), 
                    y_treino
                )
                X_treino_imagem = X_treino_imagem.reshape(-1, *TAMANHO_IMAGEM)

                print("      - Nova Distribuição lote TREINO (Reamostragem):")
                print("        -> Treino:", Counter(y_treino))
                print()

            except Exception as e:
                print("# ERRO:", str(e))
            ########################### UNDERSAMPLING ###########################

            print("   - Avaliando modelo, aguarde...")
            print("   ---")
            y_true, y_pred = [], []
            accs = []
            scores = []
            aucs = []
            suportes_relativos = []

            # Avaliando N vezes o conjunto treino/teste (consitencia)
            N = 10
            for i in tqdm_notebook(range(N)):

                # Instanciando modelo pre-treinado
                modelo = convnet_metodologia(
                    input_shape_= TAMANHO_IMAGEM,
                    output_dim = 1,
                    loss_function='binary_crossentropy',
                    metrics=['accuracy'],
                    output_activation='sigmoid'
                )

                # Treinando
                historico = modelo.fit(
                    X_treino_imagem, y_treino,
                    validation_data=(X_teste_imagem, y_teste),
                    epochs=EPOCAS,
                    batch_size=TAMANHO_LOTE,
                    #verbose=VERBOSIDADE
                    verbose=0
                )

                # Avaliando resultados
                y_teste_, y_pred_, acc, score, auc_score, suporte_relativo, idx_descartados = \
                    classificacao_threshold(
                        modelo, X_teste_imagem, y_teste, 
                        threshold={0:0, 1:0}, # TODO: validar melhor corte por aparelho
                        grafico=False
                    )


                tf.keras.backend.clear_session()

                # Incrementando resultados
                print("      -> # TENTATIVA #{}: Acurácia = {:.2f}% / F1-score = {:.2f}% / AUC = {:.3f}".format(
                    i+1, 
                    acc*100,
                    score*100,
                    auc_score,
                ))
                accs.append(acc)
                scores.append(score)
                aucs.append(auc_score)
                y_true.extend(y_teste_)
                y_pred.extend(y_pred_)

                # Guardando resultados do modelo
                resultados["aparelho"].append(rotulo_aparelho)
                resultados["iteracao"].append(iteracao) # APENAS PARA VALIDACAO CRUZADA
                resultados["teste"].append(i+1)
                resultados["acuracia"].append(acc)
                resultados["f1"].append(score)
                resultados["auc"].append(auc_score)

    print()
    print("   - Resultados finais:")
    print("   ---")
    print()
    
    print("      -> Acurácia:")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(accs)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(accs)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(accs)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(accs)*100) )
    print()

    print("      -> F1-score (macro):")
    print()
    print("         . Média geral   : {:.2f}%".format(np.mean(scores)*100 ) )
    print("         . Desvio padrão : {:.2f}%".format(np.std(scores)*100) )
    print("         . Mínimo        : {:.2f}%".format(np.min(scores)*100) )
    print("         . Máximo        : {:.2f}%".format(np.max(scores)*100) )
    print()
    print("      -> AUC:")
    print()
    print("         . Média geral   : {:.3f}".format(np.mean(aucs) ) )
    print("         . Desvio padrão : {:.3f}".format(np.std(aucs)) )
    print("         . Mínimo        : {:.3f}".format(np.min(aucs)) )
    print("         . Máximo        : {:.3f}".format(np.max(aucs)) )
    print()

    print("      -> Relatório de classificação:")
    print()
    print(classification_report(y_true, y_pred))
    print("      -> Matrix de confusao:")
    print()
    print(confusion_matrix(y_true, y_pred))
    
    print()
    print("-"*80)
    print()
    
df_rus_cv = pd.DataFrame(resultados)
df_rus_cv["metodologia"] = "RUS (Baseline + RandomUnderSampling)"
df_rus_cv.to_excel(
    os.path.join(caminho_dados_notebook, "resultados_rus_cv.xlsx"),
    index=False
)

# Análise dos Resultados (2)

## Split Treino/Teste

In [ ]:
df_baseline = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_baseline.xlsx"))
df_tce = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_tce.xlsx"))
df_classweight = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_classweight.xlsx"))
df_ros = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_ros.xlsx"))
df_rus = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_rus.xlsx"))
df_treinamento_estrategico = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_treinamento_estrategico.xlsx"))
df_transfer_learning = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_transfer_learning.xlsx"))

df_analise = pd.concat([
    # Resultados das metodologias Baseline e Final
    df_baseline, 
    df_tce,
    
    # Desbalancmeanto
    df_classweight,
    df_ros,
    df_rus,    
    df_treinamento_estrategico,
    df_transfer_learning,
])

print("* Análise por metodologia:")
df_analise_metodologia = df_analise.groupby(["metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("metodologia")
display(df_analise_metodologia)
df_analise_metodologia.to_excel(os.path.join(caminho_dados_notebook, "df_analise2_metodologia.xlsx"))

print()
print("* Análise por aparelho/metodologia:")
df_analise_aparelho = df_analise.groupby(["aparelho","metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise2_aparelho.xlsx"))

## Validação Cruzada

In [ ]:
df_baseline_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_baseline_cv.xlsx"))
df_tce_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_tce_cv.xlsx"))
df_classweight_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_classweight_cv.xlsx"))
df_ros_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_ros_cv.xlsx"))
df_rus_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_rus_cv.xlsx"))
df_treinamento_estrategico_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_treinamento_estrategico_cv.xlsx"))
df_transfer_learning_cv = pd.read_excel(os.path.join(caminho_dados_notebook, "resultados_transfer_learning_cv.xlsx"))

df_analise_cv = pd.concat([
    # Resultados das metodologias Baseline e Final
    df_baseline_cv, 
    df_tce_cv,
    
    # Desbalancmeanto
    df_classweight_cv,
    df_ros_cv,
    df_rus_cv,    
    df_treinamento_estrategico_cv,
    df_transfer_learning_cv,
])

print("* Análise por metodologia:")
df_analise_metodologia = df_analise_cv.groupby(["metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("metodologia")
display(df_analise_metodologia)
df_analise_metodologia.to_excel(os.path.join(caminho_dados_notebook, "df_analise2_metodologia_cv.xlsx"))

print()
print("* Análise por aparelho/metodologia:")
df_analise_aparelho = df_analise_cv.groupby(["aparelho","metodologia"]).agg({
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise2_aparelho_cv.xlsx"))

# Conclusões

Após várias análises, ficou evidente que:
1. A metodologia original sofre com o desbalanceeamento dos dados inerentes ao problema de NILM, principalmente para a carga analisada (Microondas, casa 3);
2. O ajuste fino do bias mais a adoção de punição de erros, parada antecipada e lote aumentado melhoram significativamente a capacidade do modelo em aprender padrões da classe minoritária (positiva, no estudo);
3. Além disso, o ajuste do limiar de decisão melhora sensivelmente a qualidade das predições, uma vez que as probabilidades do modelo são bem calibradas;
4. Por fim, a estratégia de Transfer Learning se mostrou de grande valia, corroborando as expectativas iniciais, uma vez que esta técnica permite aproveitar o máximo de conhecimento dentro de um domínio.

Sendo assim, a `metodologia final` (hipótese 6) otimizada consiste em:

- **Entrada:** RP
- **Modelo:** CNN
- **Estratégia de treinamento:**
    - Inicialização de Bias da camada de saída em função da distribuição dos dados de treinamento;
    - Adoção de fator de penalização da função de erros na classe minoritária;
    - Adoção de parada antecipada observando a métrica AUC;
    - Lote aumentado, a fim de favorecer a probabilidade de minilotes conterem amostras positivas (TODO: observar se há regra empírica).
    - Utilizar transfer learning: contemplando residências que contenham o mesmo aparelho (TODO: definir regras de modo e  sequência de utilização destes dados);
- **Pós-processamento:**
    - Limiar de Decisão nas probabilidades inferidas pelo modelo (TODO: avaliar melhor limiar);
        - Avaliar energia acumulada no descarte.

---

**PRÓXIMOS PASSOS (QUALIFICAÇÃO):**

1. Analisar tabela de resultados do estudo `17` e extrair as melhores configurações de tamanho de janela considerando:
    i. Melhor Taxa Amostral Geral;
    ii. Melhor combinação de Taxa e Janela por aparelho;
2. Encapsular Metodologia Final na classe PyNILM;
3. Avaliar a aplicação da Metodologia Final nos cenários definidos no item 1, consolidando as métricas F1, AUC e Descarte de Energia.

**IDEIAS DEFESA:**
1. Otimização arquitetura, visando Transfer Learning;
2. Capsule Networks;
3. Processamento dos descartes:
    i. One-Shot Learning;
    ii. AutoEncoder.

# Fim.

In [ ]:
%load_ext watermark

In [ ]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g